# Orchestrator (ver. 1.5)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone https://github.com/asigalov61/Orchestrator
!pip install torch
!pip install einops
!pip install torch-summary
!pip install sklearn
!pip install tqdm
!pip install matplotlib
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Orchestrator modules...')

import os
import pickle
import random
import secrets
import statistics
from time import time
import tqdm

print('=' * 70)
print('Loading main Orchestrator modules...')
import torch

%cd /content/Orchestrator

import TMIDIX

from lwa_transformer import *

%cd /content/
print('=' * 70)
print('Loading aux Orchestrator modeules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics


from midi2audio import FluidSynth
from IPython.display import Audio, display
print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

In [ ]:
#@title Unzip Pre-Trained Orchestrator Model
print('=' * 70)
%cd /content/Orchestrator/Model

print('=' * 70)
print('Unzipping pre-trained Orchestartor model...Please wait...')

!cat /content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.zip* > /content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.zip
print('=' * 70)

!unzip -j /content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.zip
print('=' * 70)

print('Done! Enjoy! :)')
print('=' * 70)
%cd /content/
print('=' * 70)

In [ ]:
#@title Load Orchestrator Pre-Trained Model
full_path_to_model_checkpoint = "/content/Orchestrator/Model/Orchestrator_Trained_Model_55253_steps_0.3277_loss.pth" #@param {type:"string"}

print('=' * 70)
print('Loading Orchestrator Pre-Trained Model...')
print('Please wait...')
print('=' * 70)
print('Instantiating model...')

SEQ_LEN = 4096

# instantiate the model

model = LocalTransformer(
    num_tokens = 774,
    dim = 1024,
    depth = 24,
    causal = True,
    local_attn_window_size = 512,
    max_seq_len = SEQ_LEN
).cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(full_path_to_model_checkpoint))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

# Plot Token Embeddings
tok_emb = model.token_emb.weight.detach().cpu().tolist()

tok_emb1 = []

for t in tok_emb:
    tok_emb1.append([abs(statistics.median(t))])

cos_sim = metrics.pairwise_distances(
   tok_emb1, metric='euclidean'
)
plt.figure(figsize=(7, 7))
plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
plt.xlabel("Position")
plt.ylabel("Position")
plt.tight_layout()
plt.plot()
plt.savefig("/content/Orchestrator-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (LOAD SEED MIDI)

In [ ]:
#@title Load Seed MIDI
select_seed_MIDI = "Orchestrator-Piano-Seed-1" #@param ["Orchestrator-Piano-Seed-1", "Orchestrator-Piano-Seed-2", "Orchestrator-Piano-Seed-3", "Orchestrator-Piano-Seed-4", "Orchestrator-Piano-Seed-5", "Orchestrator-MI-Seed-1", "Orchestrator-MI-Seed-2", "Orchestrator-MI-Seed-3", "Orchestrator-MI-Seed-4", "Orchestrator-MI-Seed-5"]
full_path_to_custom_seed_MIDI = "" #@param {type:"string"}

if full_path_to_custom_seed_MIDI == '':
  f = '/content/Orchestrator/Seeds/'+select_seed_MIDI+'.mid'

else:
  f = full_path_to_custom_seed_MIDI

print('=' * 70)
print('Orchestrator Seed MIDI Loader')
print('=' * 70)
print('Loading seed MIDI...')
print('=' * 70)
print('File:', f)
print('=' * 70)

transpose_to_model_average_pitch = False

#=======================================================
# START PROCESSING

# Convering MIDI to ms score with MIDI.py module
score = TMIDIX.midi2ms_score(open(f, 'rb').read())

# INSTRUMENTS CONVERSION CYCLE
events_matrix = []
melody_chords_f = []
melody_chords_f1 = []
itrack = 1
patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

patch_map = [[0, 1, 2, 3, 4, 5, 6, 7], # Piano 
          [24, 25, 26, 27, 28, 29, 30], # Guitar
          [32, 33, 34, 35, 36, 37, 38, 39], # Bass
          [40, 41], # Violin
          [42, 43], # Cello
          [46], # Harp
          [56, 57, 58, 59, 60], # Trumpet
          [71, 72], # Clarinet
          [73, 74, 75], # Flute
          [-1], # Drums
          [52, 53], # Choir
          [16, 17, 18, 19, 20] # Organ
          ]

while itrack < len(score):
  for event in score[itrack]:         
      if event[0] == 'note' or event[0] == 'patch_change':
          events_matrix.append(event)
  itrack += 1

events_matrix.sort(key=lambda x: x[1])

events_matrix1 = []

for event in events_matrix:
  if event[0] == 'patch_change':
      patches[event[2]] = event[3]

  if event[0] == 'note':
      event.extend([patches[event[3]]])
      once = False

      for p in patch_map:
          if event[6] in p and event[3] != 9: # Except the drums
              event[3] = patch_map.index(p)
              once = True

      if not once and event[3] != 9: # Except the drums
          event[3] = 15 # All other instruments/patches channel
          event[5] = max(80, event[5])

      if event[3] < 12: # We won't write chans 12-16 for now...
          events_matrix1.append(event)
          # stats[event[3]] += 1

#=======================================================
# PRE-PROCESSING

# checking number of instruments in a composition
instruments_list_without_drums = list(set([y[3] for y in events_matrix1 if y[3] != 9]))

instruments_list = list(set([y[3] for y in events_matrix1]))
num_instr = len(instruments_list)

# filtering out empty compositions and checking desired number of instruments in a composition
# It had been observed that music models learn best from multi-instrumental music, even for solo instruments
# So you can setup filtering by number of instruments here if you want

if len(events_matrix1) > 0 and len(instruments_list_without_drums) > 0:

    # recalculating timings
    for e in events_matrix1:
        e[1] = math.ceil(e[1] / 8) # Max 1 seconds for start-times
        e[2] = math.ceil(e[2] / 16) # Max 2 seconds for durations

    # Sorting by pitch, then by start-time
    events_matrix1.sort(key=lambda x: x[4], reverse=True)
    events_matrix1.sort(key=lambda x: x[1])

    # pitches augment stuff (calculating transpose value to C4 without drums)
    pitches = [y[4] for y in events_matrix1 if y[3] != 9]

    if len(pitches) > 0:
      avg_ptc = round(statistics.mean(pitches))
    else:
      avg_ptc = 0

    ptc_delta = 12 - (avg_ptc % 12)

    #=======================================================
    # FINAL PRE-PROCESSING

    first_event = True
    melody_chords = []
    pe = events_matrix1[0]
    pt = -1

    for e in events_matrix1:

        # Cliping all values...
        tim = max(0, min(127, e[1]-pe[1]))
        dur = max(1, min(127, e[2]))
        cha = max(0, min(11, e[3]))
        ptc = max(1, min(127, e[4]))
        vel = max(8, min(127, e[5]))

        velocity = round(vel / 15)

        # Pitches shifting
        if cha != 9:
            if transpose_to_model_average_pitch:
                ptc_aug = ptc + ptc_delta # Transposing composition to median C4
            else:
                ptc_aug = ptc
        else:
          ptc_aug = ptc + 128 # Shifting drums pitches because drums structure is different from non-drums

        # Time shifting
        if tim == 0 and first_event == True:
          abs_time = 0

        if tim == 0 and first_event == False:
          if pt == 0:
            pass
          else:
            abs_time += 128

        if tim !=0:
          abs_time = tim

        # Writing final note 
        melody_chords.append([abs_time, dur, cha, ptc_aug, velocity])

        pe = e
        pt = tim

        first_event = False

    #=======================================================
    # FINAL PROCESSING
    #=======================================================

    # Break between compositions / Intro seq

    # 758 == SOS/EOS token
    # 759 == SOS/EOS token
    # 760-761 == Composition is without drums or with drums
    # 762-773 == Number of instruments in a composition

    # TOTAL DICTIONARY SIZE OF 774 TOKENS

    if 9 in instruments_list:
      drums_present = 761 # Yes
    else:
      drums_present = 760 # No

    melody_chords_f.extend([758, 759, drums_present, 762+(num_instr-1)])
    melody_chords_f1.append([758, 759, drums_present, 762+(num_instr-1)])

    #=======================================================

    # Composition control seq
    intro_mode_time = statistics.mode([y[0] for y in melody_chords if y[2] != 9])
    intro_mode_dur = statistics.mode([y[1] for y in melody_chords if y[2] != 9])
    intro_mode_pitch = statistics.mode([y[3] for y in melody_chords if y[2] != 9])
    intro_mode_velocity = statistics.mode([y[4] for y in melody_chords if y[2] != 9])

    # Instrument value 12 is reserved for composition control seq
    intro_vel = (12 * 9) + intro_mode_velocity

    melody_chords_f.extend([intro_mode_time, intro_mode_dur+256, intro_mode_pitch+384, intro_vel+640])
    melody_chords_f1.append([intro_mode_time, intro_mode_dur+256, intro_mode_pitch+384, intro_vel+640])

    #=======================================================
    # MAIN PROCESSING CYCLE
    #=======================================================

    for m in melody_chords:

        # WRITING EACH NOTE HERE
        chan_vel = (m[2] * 9) + m[4]
        melody_chords_f.extend([m[0], m[1]+256, m[3]+384, chan_vel+640])
        melody_chords_f1.append([m[0], m[1]+256, m[3]+384, chan_vel+640])

    #=======================================================
  
    song = melody_chords_f
    song_f = []
    tim = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []
    song1 = []

    for s in song:
      if s > 256 and s < (12*9)+640:
        son.append(s)
      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
                    

    for ss in song1:

        if ss[0] < 128:
          tim += ss[0] * 8

        dur = (ss[1]-256) * 16

        if (ss[2]-384) > 128:
          pitch = (ss[2]-384) - 128
        else:
          pitch = (ss[2]-384)


        channel = (ss[3]-640) // 9
        vel = ((ss[3]-640) % 9) * 15
                        
        song_f.append(['note', tim, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Orchestrator',  
                                                          output_file_name = '/content/Orchestrator-Seed-Composition',
                                                          track_name='Project Los Angeles',
                                                          list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                          number_of_ticks_per_quarter=500)
        
    #=======================================================

print('=' * 70)
print('Composition stats:')
print('Composition has', len(melody_chords_f1), 'notes')
print('Composition has', len(melody_chords_f), 'tokens')
print('=' * 70)

print('Displaying resulting composition...')
print('=' * 70)

fname = '/content/Orchestrator-Seed-Composition'

x = []
y =[]
c = []

colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

for s in song_f:
  x.append(s[1] / 1000)
  y.append(s[4])
  c.append(colors[s[3]])

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
display(Audio(str(fname + '.wav'), rate=16000))

plt.figure(figsize=(14,5))
ax=plt.axes(title=fname)
ax.set_facecolor('black')

plt.scatter(x,y, c=c)
plt.xlabel("Time")
plt.ylabel("Pitch")
plt.show()

# (GENERATE)

In [ ]:
#@title Improv Generation

number_of_tokens_tp_generate = 512 #@param {type:"slider", min:32, max:4064, step:32}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}

#@markdown Use custom MIDI sequences if you want to have model try to mimick custom MIDI composition. Or you can specify your own, completely custom sequence for the model to use

start_improv_sequence_type = "custom_MIDI_instruments_and_composition_seq" #@param ["custom_MIDI_instruments_seq", "custom_MIDI_instruments_and_composition_seq", "user_defined_seq"]

#@markdown Specify user-defined improv sequence below

drums_present_or_not = True #@param {type:"boolean"}
number_of_instruments = 4 #@param {type:"slider", min:1, max:12, step:1}

#@markdown NOTE: If you want to specify composition seq, you must specify all four parameters. All four parameters must be > 0

desired_time = 10 #@param {type:"slider", min:0, max:127, step:1}
notes_or_chords_time = True #@param {type:"boolean"}
desired_duration = 10 #@param {type:"slider", min:0, max:127, step:1}
desired_pitch = 60 #@param {type:"slider", min:0, max:127, step:1}
instruments_or_drums_pitch = True #@param {type:"boolean"}
desired_velocity = 6 #@param {type:"slider", min:0, max:8, step:1}

#@markdown Other settings

allow_model_to_stop_generation_if_needed = False #@param {type:"boolean"}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}

print('=' * 70)
print('Orchestrator Improv Model Generator')
print('=' * 70)

if allow_model_to_stop_generation_if_needed:
  min_stop_token = 758
else:
  min_stop_token = 0

if start_improv_sequence_type == 'custom_MIDI_instruments_seq':
  outy = melody_chords_f[:4]

if start_improv_sequence_type == 'custom_MIDI_instruments_and_composition_seq':
  outy = melody_chords_f[:8]

if start_improv_sequence_type == 'user_defined_seq':
  if drums_present_or_not:
    drumsp = 1
  else:
    drumsp = 0

  outy = [758, 759, 760+drumsp, 762+((number_of_instruments)-1)]
  test_info = [desired_time, desired_duration, desired_pitch, desired_velocity]

  if 0 not in test_info:

    if not notes_or_chords_time:
      desired_time += 128
    
    if not instruments_or_drums_pitch:
      desired_pitch += 128

    outy.extend([desired_time, desired_duration+256, desired_pitch+384, ((12*9)+desired_velocity)+640])

print('Selected Improv sequence:')
print(outy)
print('=' * 70)

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

start_time = time()

out = model.generate(inp, 
                      number_of_tokens_tp_generate, 
                      temperature=temperature, 
                      return_prime=True, 
                      min_stop_token=min_stop_token, 
                      verbose=True)

out0 = out.tolist()

print('=' * 70)
print('Done!')
print('=' * 70)
print('Generation took', time() - start_time, "seconds")
print('=' * 70)

#======================================================================

print('Rendering results...')
print('=' * 70)

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:12])
  print('=' * 70)

  if len(out) != 0:
      
      song = out1
      song_f = []
      tim = 0
      dur = 0
      vel = 0
      pitch = 0
      channel = 0

      son = []
      song1 = []

      for s in song:
        if s > 256 and s < (12*9)+640:
          son.append(s)
        else:
          if len(son) == 4:
            song1.append(son)
          son = []
          son.append(s)
                      

      for ss in song1:

          if ss[0] < 128:
            tim += ss[0] * 8

          dur = (ss[1]-256) * 16

          if (ss[2]-384) > 128:
            pitch = (ss[2]-384) - 128
          else:
            pitch = (ss[2]-384)


          channel = (ss[3]-640) // 9
          vel = ((ss[3]-640) % 9) * 15
                          
          song_f.append(['note', tim, dur, channel, pitch, vel ])

      detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Orchestrator',  
                                                          output_file_name = '/content/Orchestrator-Music-Composition_'+str(i), 
                                                          track_name='Project Los Angeles',
                                                          list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],                                                        number_of_ticks_per_quarter=500)
      
      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Orchestrator-Music-Composition_'+str(i)

      x = []
      y =[]
      c = []

      colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

      for s in song_f:
        x.append(s[1] / 1000)
        y.append(s[4])
        c.append(colors[s[3]])

      FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
      display(Audio(str(fname + '.wav'), rate=16000))

      plt.figure(figsize=(14,5))
      ax=plt.axes(title=fname)
      ax.set_facecolor('black')

      plt.scatter(x,y, c=c)
      plt.xlabel("Time")
      plt.ylabel("Pitch")
      plt.show() 

In [ ]:
#@title Standard/Simple Continuation

number_of_prime_tokens = 512 #@param {type:"slider", min:4, max:3072, step:4}
number_of_tokens_tp_generate = 512 #@param {type:"slider", min:32, max:4096, step:32}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
include_prime_tokens_in_generated_output = True #@param {type:"boolean"}
allow_model_to_stop_generation_if_needed = True #@param {type:"boolean"}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}

print('=' * 70)
print('Orchestrator Standard Model Generator')
print('=' * 70)

if allow_model_to_stop_generation_if_needed:
  min_stop_token = 758
else:
  min_stop_token = 0

outy = melody_chords_f[:number_of_prime_tokens]

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

start_time = time()

out = model.generate(inp, 
                      number_of_tokens_tp_generate, 
                      temperature=temperature, 
                      return_prime=include_prime_tokens_in_generated_output, 
                      min_stop_token=min_stop_token, 
                      verbose=True)

out0 = out.tolist()
print('=' * 70)
print('Done!')
print('=' * 70)
print('Generation took', time() - start_time, "seconds")
print('=' * 70)
#======================================================================
print('Rendering results...')
print('=' * 70)

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:12])
  print('=' * 70)

  if len(out) != 0:
      
      song = out1
      song_f = []
      tim = 0
      dur = 0
      vel = 0
      pitch = 0
      channel = 0

      son = []
      song1 = []

      for s in song:
        if s > 256 and s < (12*9)+640:
          son.append(s)
        else:
          if len(son) == 4:
            song1.append(son)
          son = []
          son.append(s)
                      

      for ss in song1:

          if ss[0] < 128:
            tim += ss[0] * 8

          dur = (ss[1]-256) * 16

          if (ss[2]-384) > 128:
            pitch = (ss[2]-384) - 128
          else:
            pitch = (ss[2]-384)


          channel = (ss[3]-640) // 9
          vel = ((ss[3]-640) % 9) * 15
                          
          song_f.append(['note', tim, dur, channel, pitch, vel ])

      detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Orchestrator',  
                                                          output_file_name = '/content/Orchestrator-Music-Composition_'+str(i), 
                                                          track_name='Project Los Angeles',
                                                          list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],                                                        number_of_ticks_per_quarter=500)
      
      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Orchestrator-Music-Composition_'+str(i)

      x = []
      y =[]
      c = []

      colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

      for s in song_f:
        x.append(s[1] / 1000)
        y.append(s[4])
        c.append(colors[s[3]])

      FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
      display(Audio(str(fname + '.wav'), rate=16000))

      plt.figure(figsize=(14,5))
      ax=plt.axes(title=fname)
      ax.set_facecolor('black')

      plt.scatter(x,y, c=c)
      plt.xlabel("Time")
      plt.ylabel("Pitch")
      plt.show() 

In [ ]:
#@title Stabilized Continuation

#@markdown NOTE: Play with the settings to find optinal ones for your seed MIDI

#@markdown NOTE: Increasing number of batches should increase stability/quality of the output but it will slow down the generation proportionally

number_of_prime_tokens = 512 #@param {type:"slider", min:4, max:3072, step:4}
block_size_in_tokens = 4 #@param {type:"slider", min:1, max:64, step:1}
number_of_blocks_to_generate = 128 #@param {type:"slider", min:1, max:512, step:1}
number_of_batches_per_block = 2 #@param {type:"slider", min:2, max:16, step:1}
include_prime_tokens_in_generated_output = True #@param {type:"boolean"}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}

def closest_acc_index(cba, acc):
    return min(range(len(cba)), key = lambda i: abs(cba[i]-acc))

print('=' * 70)
print('Orchestrator Stabilized Model Output Generator')
print('=' * 70)
print('Generating...')
print('=' * 70)

outy = melody_chords_f[:number_of_prime_tokens]

inp = [outy] * number_of_batches_per_block
inp = torch.LongTensor(inp).cuda()
baseline_acc = model.choose_best_acc(inp)[0][2]

if not include_prime_tokens_in_generated_output:
  outy = []

start_time = time()

stats = []

for i in tqdm.tqdm(range(number_of_blocks_to_generate)):

    out = model.generate(inp, 
                         block_size_in_tokens, 
                         temperature=temperature, 
                         return_prime=True,  
                         verbose=False)
    
    cba = model.choose_best_acc(out)
    acc_batch_number = closest_acc_index([y[2] for y in cba], baseline_acc)
    cba_idx = cba[acc_batch_number][0]
    stats.append(cba)
    
    inp = out[cba_idx].tolist()
    outy.extend(inp[-block_size_in_tokens:])
    inp = [inp] * number_of_batches_per_block
    
    inp = torch.LongTensor(inp).cuda()

print('=' * 70)
print('Generation took', time() - start_time, "seconds")

print('=' * 70)
print('Composition stats:')
print('Prime/Baseline accuracy:', baseline_acc)
print('Final/Generated accuracy', cba[acc_batch_number][2])
print('=' * 70)

#======================================================================

out1 = outy

print('Sample INTs', out1[:12])
print('=' * 70)

if len(out) != 0:
    
    song = out1
    song_f = []
    tim = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []
    song1 = []

    for s in song:
      if s > 256 and s < (12*9)+640:
        son.append(s)
      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
                    

    for ss in song1:

        if ss[0] < 128:
          tim += ss[0] * 8

        dur = (ss[1]-256) * 16

        if (ss[2]-384) > 128:
          pitch = (ss[2]-384) - 128
        else:
          pitch = (ss[2]-384)


        channel = (ss[3]-640) // 9
        vel = ((ss[3]-640) % 9) * 15
                        
        song_f.append(['note', tim, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Orchestrator',  
                                                        output_file_name = '/content/Orchestrator-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

print('=' * 70)
print('Displaying resulting composition...')
print('=' * 70)

fname = '/content/Orchestrator-Music-Composition'

x = []
y =[]
c = []

colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

for s in song_f:
  x.append(s[1] / 1000)
  y.append(s[4])
  c.append(colors[s[3]])

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
display(Audio(str(fname + '.wav'), rate=16000))

plt.figure(figsize=(14,5))
ax=plt.axes(title=fname)
ax.set_facecolor('black')

plt.scatter(x,y, c=c)
plt.xlabel("Time")
plt.ylabel("Pitch")
plt.show()  

# (ORCHESTRATE)

In [ ]:
#@title Orchestration
use_customized_first_note = True #@param {type:"boolean"}
first_note_instrument = "Clarinet" #@param ["Piano", "Guitar", "Bass", "Violin", "Cello", "Harp", "Trumpet", "Clarinet", "Flute", "Choir", "Organ"]
first_note_velocity = 7 #@param {type:"slider", min:1, max:8, step:1}
composition_orchestration_length_in_notes = 256 #@param {type:"slider", min:8, max:1024, step:4}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}

#======================================

instruments_list = ["Piano", "Guitar", "Bass", "Violin", "Cello", "Harp", "Trumpet", "Clarinet", "Flute", 'Drums', "Choir", "Organ"]
start_instrument_number = instruments_list.index(first_note_instrument)

print('=' * 90)
print('Orchestrator Composition Instrumentation Generator')
print('=' * 90)

start_time = time()

if use_customized_first_note:
  inp = melody_chords_f[:11] + [((start_instrument_number*9)+first_note_velocity)+640] + melody_chords_f[12:15]
else:
  inp = melody_chords_f[:11]

start_note = int((len(inp)+1) / 4)

inp = torch.LongTensor(inp).cuda()

for i in tqdm.tqdm(range(start_note, len(melody_chords_f1[:composition_orchestration_length_in_notes]))):

    out = model.generate(inp[None, ...], 
                         1, 
                         temperature=temperature, return_prime=False, verbose=False)
    
    inp = torch.cat((inp, out[0]))
    
    next_note = torch.LongTensor(melody_chords_f1[i][:3]).cuda()
    
    inp = torch.cat((inp, next_note))

print('=' * 90)
print('Generation took', time() - start_time, "seconds")
print('=' * 90)
print('Composition stats:')
print('Loss:', model.choose_best_acc([inp[-SEQ_LEN:]])[0][1])
print('Accuracy:', model.choose_best_acc([inp[-SEQ_LEN:]])[0][2])
print('=' * 90)

#======================================================================

out1 = inp.tolist()

print('Sample INTs', out1[:16])
print('=' * 90)

if len(out) != 0:
    
    song = out1
    song_f = []
    tim = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []
    song1 = []

    for s in song:
      if s > 256 and s < (12*9)+640:
        son.append(s)
      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
                    

    for ss in song1:

        if ss[0] < 128:
          tim += ss[0] * 8

        dur = (ss[1]-256) * 16

        if (ss[2]-384) > 128:
          pitch = (ss[2]-384) - 128
        else:
          pitch = (ss[2]-384)


        channel = (ss[3]-640) // 9
        vel = ((ss[3]-640) % 9) * 15
                        
        song_f.append(['note', tim, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Orchestrator',  
                                                        output_file_name = '/content/Orchestrator-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

print('=' * 90)

#===================================================

print('Displaying resulting composition...')
print('=' * 90)

fname = '/content/Orchestrator-Music-Composition'

x = []
y =[]
c = []

colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver']

for s in song_f:
  x.append(s[1] / 1000)
  y.append(s[4])
  c.append(colors[s[3]])

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
display(Audio(str(fname + '.wav'), rate=16000))

plt.figure(figsize=(14,5))
ax=plt.axes(title=fname)
ax.set_facecolor('black')

plt.scatter(x,y, c=c)
plt.xlabel("Time")
plt.ylabel("Pitch")
plt.show()

# Congrats! You did it! :)